A. REGRESSION

In [1]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

/Users/See/anaconda/envs/ML/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = quandl.get("WIKI/GOOGL")
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [3]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close','Adj. Volume']]
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume
Date                                                                
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0


In [4]:
#Percentage between High and Low - to understand vol
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100

In [5]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

In [6]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.072956    0.324968   44659000.0
2004-08-20   54.322689  7.921706    7.227007   22834300.0
2004-08-23   54.869377  4.049360   -1.227880   18256100.0
2004-08-24   52.597363  7.657099   -5.726357   15247300.0
2004-08-25   53.164113  3.886792    1.183658    9188600.0


In [7]:
#forecasting Adjusted Close price
forecast_col = 'Adj. Close'
##Machine Learning cannot handle N/A
df.fillna(-99999, inplace=True)

In [8]:
#math.ceil rounds up decimal numbers, and returns a float, hence needed a int
#try to predict price in 10 days
forecast_out = int(math.ceil(0.01*len(df)))

In [9]:
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
print(df.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
Date                                                              
2017-09-12      946.65  1.118153   -0.028513    1245767.0  1033.13
2017-09-13      950.44  0.853289    0.522475    1092849.0  1033.04
2017-09-14      940.13  1.029007   -0.620507    1415168.0  1042.60
2017-09-15      935.29  1.122646   -0.510589    1940047.0  1042.97
2017-09-18      929.75  1.232589   -0.562561    1445532.0  1049.99


In [10]:
#X denotes Features
#y denotes Labels

X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

#scaling (skip for high frequency)
X = preprocessing.scale(X)

#X = X[:-forecast_out+1]
df.dropna(inplace=True)
y=np.array(df['label'])

print(len(X), len(y))

3294 3294


In [11]:
#use 20% of the data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

In [12]:
#classifier
clf = LinearRegression()
#can switch between algorithms
##support vector machine
#clf = svm.SVR()
#clf = svm.SVR(kernel='poly')
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
#train data needs to be seperated from test data otherwise, 
#the answer would be known already
print(accuracy)

0.974178083893


In [13]:
print(forecast_out)

34
